In [ ]:
import sonata
import torch
import time

point = sonata.data.load("sample1")
print(f"Point data loaded with {len(point)} points.")
transform = sonata.transform.default()
model = sonata.model.load("sonata", repo_id="facebook/sonata").cuda()

for param in model.parameters():
    param.requires_grad = False
      
start_time = time.time()

point = transform(point)
for key in point.keys():
    if isinstance(point[key], torch.Tensor):
        point[key] = point[key].cuda(non_blocking=True)
with torch.no_grad():
    point = model(point)

for _ in range(2):
    assert "pooling_parent" in point.keys()
    assert "pooling_inverse" in point.keys()
    parent = point.pop("pooling_parent")
    inverse = point.pop("pooling_inverse")
    parent.feat = torch.cat([parent.feat, point.feat[inverse]], dim=-1)
    point = parent
while "pooling_parent" in point.keys():
    assert "pooling_inverse" in point.keys()
    parent = point.pop("pooling_parent")
    inverse = point.pop("pooling_inverse")
    parent.feat = point.feat[inverse]
    point = parent
feat = point.feat[point.inverse]

print(f"Time taken: {time.time() - start_time:.4f} seconds")
print(point['feat'].shape)
print(point['coord'].shape)